In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer, KNNImputer, MissingIndicator

from functions import *

pd.set_option('display.max_columns', 100)

# load

In [2]:
app_train = pd.read_csv('processed-data/processed_application_train.csv')
app_test = pd.read_csv('processed-data/processed_application_test.csv')

In [3]:
app_train.set_index('SK_ID_CURR', inplace=True)
app_test.set_index('SK_ID_CURR', inplace=True)

In [4]:
app_train.head()

,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_2,EXT_SOURCE_3,FLOORSMAX_AVG,FLOORSMAX_MODE,FLOORSMAX_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_3,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_8,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,OWN_CAR_AGE_missing_flag,OCCUPATION_TYPE_missing_flag,EXT_SOURCE_1_missing_flag,...,ORGANIZATION_TYPE_Industry: type 3,ORGANIZATION_TYPE_Industry: type 9,ORGANIZATION_TYPE_Kindergarten,ORGANIZATION_TYPE_Medicine,ORGANIZATION_TYPE_Other,ORGANIZATION_TYPE_School,ORGANIZATION_TYPE_Security,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Trade: type 3,ORGANIZATION_TYPE_Trade: type 7,ORGANIZATION_TYPE_Transport: type 4,ORGANIZATION_TYPE_nan,FONDKAPREMONT_MODE_org spec account,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_reg oper spec account,FONDKAPREMONT_MODE_nan,HOUSETYPE_MODE_block of flats,HOUSETYPE_MODE_nan,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_nan,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_nan,DOCUMENT_COUNT,NEW_DOC_KURT,AGE_RANGE,EXT_SOURCES_MIN,EXT_SOURCES_MAX,EXT_SOURCES_MEAN,EXT_SOURCES_NANMEDIAN,DAYS_EMPLOYED_PERC,INCOME_CREDIT_PERC,INCOME_PER_PERSON,CREDIT_TO_GOODS_RATIO,INCOME_TO_BIRTH_RATIO,ID_TO_BIRTH_RATIO,APPS_EXT_SOURCE_MEAN,APP_SCORE2_TO_REGION_RATING_RATIO,APP_SCORE2_TO_CITY_RATING_RATIO,APP_SCORE2_TO_POP_RATIO,APP_EXT_SOURCE_2*EXT_SOURCE_3,APP_EXT_SOURCE_2*DAYS_EMPLOYED,APP_EXT_SOURCE_3*DAYS_EMPLOYED,APPS_GOODS_INCOME_RATIO,APPS_CNT_FAM_INCOME_RATIO,CREDIT_TO_GOODS_RATIO_2,APP_AMT_INCOME_TOTAL_12_AMT_ANNUITY_ratio,APP_DAYS_EMPLOYED_DAYS_BIRTH_diff
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
278621,0,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,-16765,-1188.0,-1186.0,-291,1,0,1,0,2.0,1,1,11,0,0,0,0,0,0,0.622246,NaN,0.2917,0.2917,0.2917,0.0714,1.0,0.0,1.0,0.0,-828.0,1,0,0,0,0.0,0.0,0.0,0.0,0.0,True,False,False,...,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,True,False,True,False,False,False,False,True,False,1,20.0,3,0.311267,0.622246,0.466757,0.466757,0.070862,0.208736,135000.0,1.145199,-16.104981,0.017358,0.466757,0.622246,0.622246,175.726003,NaN,-739.227981,NaN,4.183333,135000.0,1.145199,-13198.5,15577.0
139008,0,0,0,1,0,135000.0,312682.5,29686.5,297000.0,-19005,-3039.0,-9833.0,-2437,1,0,0,0,2.0,2,2,17,0,0,0,0,0,0,0.650442,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2.0,0.0,-617.0,1,0,0,0,NaN,NaN,NaN,NaN,NaN,True,False,True,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,True,False,True,1,20.0,4,0.650442,0.650442,0.650442,0.650442,0.159905,0.431748,67500.0,1.052803,-7.103394,0.128229,0.650442,0.325221,0.325221,81.112569,NaN,-1976.692297,NaN,2.200000,67500.0,1.052803,-18436.5,15966.0
138348,0,1,0,1,0,121500.0,513000.0,21865.5,513000.0,-19932,-3038.0,-4311.0,-3458,1,0,0,0,1.0,2,2,11,0,0,0,0,1,1,0.322738,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-1106.0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,True,False,True,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,True,False,True,1,20.0,4,0.322738,0.322738,0.322738,0.322738,0.152418,0.236842,121500.0,1.000000,-6.095725,0.173490,0.322738,0.161369,0.161369,11.25975

# merge

# train

In [5]:
X, y = app_train.drop(['TARGET'], axis=1), app_train['TARGET']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=64)

# replace inf values with nan
X_train = X_train.replace([np.inf, -np.inf], np.nan)
X_valid = X_valid.replace([np.inf, -np.inf], np.nan)

# robust scaler
robust_scaler = RobustScaler(quantile_range=(0, 98))
robust_scaler.fit(X_train)
X_train = robust_scaler.transform(X_train)
X_valid = robust_scaler.transform(X_valid)

# scale values
minmax_scaler = MinMaxScaler(feature_range=(0, 1))
minmax_scaler.fit(X_train)
X_train = minmax_scaler.transform(X_train)
X_valid = minmax_scaler.transform(X_valid)

In [6]:
# fill missing values with median
imputer = SimpleImputer(strategy='mean')
imputer.fit(X_train)
X_train = imputer.transform(X_train)
X_valid = imputer.transform(X_valid)

In [7]:
# train model
logreg = LogisticRegression(class_weight='balanced', solver='newton-cholesky')
logreg.fit(X_train, y_train)

# predict
y_pred = logreg.predict_proba(X_valid)[:, 1]

# evaluate
roc = roc_auc_score(y_valid, y_pred)
print('ROC AUC: {:.4f}'.format(roc))
print('GINI: {:.4f}'.format(2*roc - 1))

ROC AUC: 0.7464
GINI: 0.4928


# test

In [8]:
# reorder columns to match train
app_test_clean = app_test[X.columns]

# replace inf values with nan
X_test = app_test_clean.replace([np.inf, -np.inf], np.nan)

# robust scaler
X_test = robust_scaler.transform(X_test)

# scale values
X_test = minmax_scaler.transform(X_test)

# fill missing values with median
X_test = imputer.transform(X_test)

# predict
y_pred = logreg.predict_proba(X_test)[:, 1]

# submission
submission = pd.DataFrame({'SK_ID_CURR': app_test_clean.index, 'TARGET': y_pred})

submission.to_csv('submission.csv', index=False)

submission.head()

,SK_ID_CURR,TARGET
0,83659,0.782478
1,174814,0.511729
2,179486,0.796951
3,57038,0.095772
4,25672,0.334438
